### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain_groq]


In [5]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1373ff5b0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1373fd600>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to hindi."),
    HumanMessage(content="how are you?"),]

result=model.invoke(messages)

In [10]:
# Output parser

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(result)

'आप कैसे हैं? (aap kaise hain?) \n'

In [12]:
### Using LCEL- chain the components
chain=model|output_parser
chain.invoke(messages)

'आप कैसे हैं? (aap kaise hain?) \n'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [14]:
result=prompt.invoke({"language":"Hindi","text":"How are you?"})

In [15]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]

In [17]:
##Chaining together components with LCEL
chain=prompt|model|output_parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'